## Klassifikation der einzelnen Objecte im mvtec_anomaly_detection Dataset 

### Importieren der Bibliotheken und Files 

In [8]:
from read_and_label_Images_only_objects import read_and_label_Images_only_objects
from read_and_label_Images import read_and_label_Images
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf


In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
src_path="orginal_Dataset/mvtec_anomaly_detection/"
dst_path="Set_01/"
dst_path_2="Set_02/"
dst_path_3="Dataset/Set_02/"

#read_and_label_Images_only_objects(src_path,dst_path)
#read_and_label_Images(src_path,dst_path_2)
read_and_label_Images(src_path,dst_path_3)